## LSTM
### 框架
Long short-term memory 主要是为了解决RNN<font color='red'>长期依赖</font>的问题,LSTM最重要的概念在与引入了cell state的概念
1. 输入门： 决定是否接收当前的输入
2. 遗忘门： 决定是否遗忘上一时刻的hidden
3. 输出门： 决定是否output当前的输出
<img src="./image/LSTM.png" style="zoom:40%">

### 定义
为了更好地说明LSTM的计算过程，首先定义一些符号：<br>
$x_{t}表示当前t时刻的输入$ <br>
$W_{i},W_{f},W_{o}表示的是三个门的权重矩阵，和x_{t}进行运算$ <br>
$W_{c}表示的是cell的权重矩阵，和x_{t}进行运算$<br>
$U_{i},U_{f},U_{o}表示的是三个门的权重矩阵，和h_{t-1}进行运算$ <br>
$U_{c}表示的是cell的权重矩阵，和h_{t-1}进行运算$<br>
$V_{o}表示的是输出门的权重矩阵，和c_{t}进行运算$<br>


### 计算过程
#### 输入门 input gate
输入门的状态有两部分确定，第一部分是当前的输入 $x_{t}$，第二部分是上次输出的 $h_{t-1}$。<br>
候选的memory cell也由上面的两部分确定,候选的原因在于输入门的控制使得memory cell不一定能够输入。
$$输入门的状态：i_t = \sigma (W_ix_t+U_{i}h_{t-1}+b_{i})$$
$$候选的memory\;cell：\widetilde{C}_t = tanh(W_cx_t+U_ch_{t-1}+b_{c}) $$

#### 遗忘门 forget gate
遗忘门也由两部分确定，第一部分是当前的输入 $x_{t}$，第二部分是上次输出的 $h_{t-1}$。<br>
$$f_t = \sigma (W_fx_t+U_fh_{t-1}+b_f)$$

#### 计算memory cell ，这里的计算时乘法，<font color = 'red'> 也就是元素乘以对应的元素 </font>
在拥有了输入门和遗忘门，以及候选的memory cell之后，可以决定当前的memory cell的数值：
当前的memory cell由四个部分组成：输入门，遗忘门，当前输入构成的候选memory cell，上一时刻的memory cell
$$C_t = i_t*\widetilde{C}_t + f_t * C_{t-1}$$


#### 计算输出门
输出门是由三个部分组成：1. 当前的输入 $x_{t}$  2. 上一次输入的 $h_{t-1}$ 3. 当前的memory cell $C_t$的状态<br>
而新的隐藏层则由输出门和当前的memory cell $C_t$决定 <br>
$$o_t = \sigma(W_ox_t+U_oh_{t-1}+ V_oC_t + b_o)$$
$$h_t = o_t * tanh(C_t)$$

#### 门的操作
这是一个连续的操作<br>
The sigmoid layer outputs numbers between zero and one, describing <font color='red'>how much of each component</font> should be let through. A value of zero means “let nothing through,” while a value of one means “let everything through!”

In [2]:
import numpy as np
import matplotlib.pyplot as plt

In [4]:
data = open("input.txt",'r').read()

In [11]:
chars = list(set(data))
data_size, X_size = len(data),len(chars)
print("data has %d chars, %d unique" % (data_size,X_size))
char_to_idx = {ch:i for i,ch in enumerate(chars)}
idx_to_char = {i:ch for i,ch in enumerate(chars)}

data has 99993 chars, 62 unique


In [13]:
H_size = 100
learning_rate = 1e-1

T_steps = 25
weight_sd = 0.1
z_size = H_size + X_size # 表示将上次的hidden与当前的输入结合在一起的size

In [15]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def dsigmoid(y):
    return y * (1 - y)

def tanh(x):
    return np.tanh(x)

def dtanh(y):
    return 1 - y * y

# Param
## 输入
$X_t$ : 当前的输入 $X_t$ 是一个包含n个特征的一行数据，它的特征的大小可以表示为 $X_{1\times n}$<br>
$h_{t-1}$:上一时刻的隐藏层$h_{t-1}$的大小取决与隐藏层设置的大小，比如设置为$m$，那么$h_{t-1}$的大小可以表示为$h_{1\times m}$<br>
$Z_t$: $X_t$ 和 $h_{t-1}$ 合并得到的结果，大小可以表示为 $Z_{1\times (m+n)}$ <br>
$C_{t-1}$: 记忆单元的尺寸，和$h_{t-1}$尺寸相同，$C_{1 \times m}$

## 矩阵
$W_f$: 遗忘门矩阵，大小可以表示为 $W_{m \times (m+n)}$
$W_i$: 输入门矩阵，大小可以表示为 $W_{m \times (m+n)}$
$W_o$: 输出门矩阵，大小可以表示为 $W_{m \times (m+n)}$
$W_f$: Memory cell 运算矩阵，大小可以表示为 $W_{m \times (m+n)}$

## 输出
$W_v$: 输出运算矩阵，假设输出$q$个类，$h_t$的大小为m,那么$W$的矩阵可以表示为$W_{m \times q}$

In [24]:
class Param:
    def __init__(self,name,value):
        self.name = name
        self.v = value
        self.d = np.zeros_like(value) # 导数
        self.m = np.zeros_like(value) # 动量 for AdaGard

In [33]:
class Parameters:
    def __init__(self):
        # 遗忘门
        self.W_f = Param('W_f',np.random.randn(H_size,z_size) * weight_sd + 0.5)
        self.b_f = Param('b_f',np.zeros((H_size,1)))
        
        # 输入门
        self.W_i = Param('W_i',np.random.randn(H_size,z_size) * weight_sd + 0.5)
        self.b_i = Param('b_i',np.zeros((H_size,1)))

        # 输出门
        self.W_o = Param('W_o',np.random.randn(H_size,z_size) * weight_sd + 0.5)
        self.b_o = Param('b_o',np.zeros((H_size,1)))
        
        self.W_C = Param('W_C',np.random.randn(H_size,z_size) * weight_sd)
        self.b_C = Param('b_C',np.zeros((H_size,1)))
        
        self.W_v = Param('W_v',np.random.randn(X_size,H_size) *  weight_sd)
        self.b_v = Param('b_v',np.zeros((X_size,1)))
    
    def all(self):
        return [self.W_f,self.W_i,self.W_C,self.W_o,self.W_v,
               self.b_f,self.b_i,self.b_C,self.b_o,self.b_v]

In [40]:
params  = Parameters()

# 前向传播 forward

In [42]:
def forward(x,h_prev,C_prev,p=params):
    """
    x,h_prev,C_prev依次表示当前的输入，上一时刻隐藏元，上一时刻内存单元
    x.shape = x_size,1
    h_prev = H_size,1
    C_prev = H_size,1
    """
    
    z = np.row_stack((h_prev,x)) # (x_size + H_size = z_size) * 1
    
    f = sigmoid(np.dot(p.W_f,z) + p.b_f)
    i = sigmoid(np.dot(p.W_i,z) + p.b_f)
    o = sigmoid(np.dot(p.W_o,z) + p.b_f)
    
    C_bar = tanh(np.dot(p.W_C,z) + p.b_c)
    C = i * C_bar + f * C_prev
    
    h_t = o * tanh(C)
    
    v = np.dot(p.W_v,h_t) + p.b_v
    
    
    y = np.exp(v) / np.exp(v).sum()
    
    return y

# 反向传播 backward

1. 损失函数分为两个部分：第一部分时在$t$时刻产生的输出$y_t$造成的误差，第二部分时输出的每个类$\hat{y}_j$与真实结果之间的误差

In [ ]:
def backward(target,dh_next,dC_next,C_prev,):